In [ ]:
from dataclasses import dataclass
import typer

app = typer.Typer()
@dataclass
class Common:
    profile_dir: str
        

@app.callback()
def profile_callback(ctx: typer.Context,
                     profile_dir: str = typer.Option(..., )):

    typer.echo(f"Hello {profile_dir}")

    test_dir = "hello"

    """Common Entry Point"""
    ctx.obj = Common(ctx)

    return (test_dir)

print(ctx.obj.a)


In [ ]:
class Config(object):

    def __init__(self, config):
        # do something with config here ...
        self.a = 'example_A'
        self.b = 'example_B'

# in your def cli()
ctx.obj = Config(config)
# in your commands you can works with your prepared data like this:
print(ctx.obj.a)
print(ctx.obj.b)

In [ ]:
def profile_callback(ctx,profile_dir):
    
    print (ctx + " " + profile_dir)

cli_profile_path = profile_callback("lewis","baker")

    
    

In [ ]:
def my_function(fname, lname):
  print(fname + " " + lname)

my_function("Lewis", "Baker")


In [ ]:
def func():
  func.variable = "John"

func()

print(func.variable)

In [ ]:
@dataclass
class Common:

    profile_dir: str

    @app.callback()
    def profile_callback(ctx: typer.Context,
                        profile_dir: str = typer.Option(..., )):

        typer.echo(f"Hello {profile_dir}")

        path = (f"{profile_dir}")

        ctx.obj = Common(profile_dir)

        return path


profile_path = Common()

##profile_path = Common("profile_dir")

cli_profile_path = (profile_path.profile_dir)

In [ ]:
@dataclass
class Common:

    #profile_dir: str
    root = str

    @app.callback()
    def profile_callback(##ctx: typer.Context,
                        profile_dir: str = typer.Option(..., )):
        
        #typer.echo(f"Hello {profile_dir}")
        
        print("I'm a", profile_dir.root)
        
        ##ctx.obj = Common(profile_dir)


profile_path = Common()

##profile_path = Common("profile_dir")

print(profile_path.root)
profile_path.profile_callback()

In [7]:
# A Sample class with init method
class Person:
   
    # init method or constructor 
    def __init__(self, name):
        self.name = name
   
    # Sample Method 
    def say_hi(self):
        print('Hello, my name is', self.name)
   
    def assignment():
    
        p = Person('Nikhil')
    
        p.say_hi()
        
Person.assignment()

Hello, my name is Nikhil


In [29]:
from dataclasses import dataclass
import typer

app = typer.Typer()


@dataclass
class Common:

    def __init__(profile_dir, name):
        profile_dir.name = name

    @app.callback()
    def profile_callback(##ctx: typer.Context,
                        profile_dir: str = typer.Option(..., )):
        
        print('Hello, my name is', profile_dir.name)
        
        ##path = self.profile_dir
        
        ##ctx.obj = Common(profile_dir)

        
    def assignment():
    
        p = Common('directory')
        p.profile_callback()

Common.assignment()


Hello, my name is directory


In [48]:
from dataclasses import dataclass
import typer

import os
import yaml

app = typer.Typer()


@dataclass
class Common:

    def __init__(profile_dir, name):
        profile_dir.name = name

    @app.callback()
    def profile_callback(##ctx: typer.Context,
                        profile_dir: str = typer.Option(..., )):
        
        print('Hello, my name is', profile_dir.name)
        
        ##path = self.profile_dir
        
        ##ctx.obj = Common(profile_dir)

        
    def assignment():
    
        p = Common('directory')
        p.profile_callback()


Common.assignment()




Hello, my name is directory


In [57]:
from dataclasses import dataclass
import typer

import os
import yaml

app = typer.Typer()


@dataclass
class Common:

    def __init__(profile_dir, lookml_config, name):
        profile_dir.name = name
        lookml_config.name = name

    @app.callback()
    def profile_callback(ctx: typer.Context,
                        profile_dir: str = typer.Option(..., ),
                        lookml_config: str
                        ):
        
        print(f"Hello, my name is {profile_dir}")

        ctx.obj = Common(profile_dir)


        path = {profile_dir}

        profile_pass = os.path.join(path,"profile.yaml")

        with open(profile_pass) as f:
            lookml_config = yaml.load(f, Loader=yaml.FullLoader)

    def assignment(profile_dir: str = typer.Option(..., )):
    
        p = (f" assignment test {profile_dir}")
        print (p)
        return p
    
cli_profile_path = Common.assignment()



 assignment test <typer.models.OptionInfo object at 0x10dff5580>


In [49]:
class Dog:
     
    # A simple class
    # attribute
    attr1 = "mammal"
    attr2 = "dog"
 
    # A sample method 
    def fun(self):
        print("I'm a", self.attr1)
        print("I'm a", self.attr2)
 
# Driver code
# Object instantiation
Rodger = Dog()
 
# Accessing class attributes
# and method through objects
print(Rodger.attr1)
Rodger.fun()

mammal
I'm a mammal
I'm a dog


In [69]:
from dataclasses import dataclass
import typer

import os
import yaml

app = typer.Typer()


@dataclass
class Common:

    def __init__(profile_dir, name):
        profile_dir.name = name

    @app.callback()
    def profile_callback(##ctx: typer.Context,
                        profile_dir: str = typer.Option(..., )):
        
        print('Hello, my name is', profile_dir.name)
        
        test = Common("test")
        
        ##path = self.profile_dir
        
        ##ctx.obj = Common(profile_dir)

        
    def assignment():
    
        p = Common('directory')
        p.profile_callback()


Common.assignment()

Rodger = Common()

print (Rodger.profile_dir)


Hello, my name is directory


TypeError: __init__() missing 1 required positional argument: 'name'

In [90]:
from dataclasses import dataclass
import typer

import os
import yaml

app = typer.Typer()


@dataclass
class Common:

    profile_dir: str

    @app.callback()
    def profile_callback(##ctx: typer.Context,
                        profile_dir: str = typer.Option(..., )):
        
        print('Hello, my name is', profile_dir)
        
        test = Common("test")
        
        ##p = Common('directory')
        ##p.profile_callback()
        
        ##path = self.profile_dir
        
        ##ctx.obj = Common(profile_dir)

        
    def assignment():
    
        p = Common('directory')
        p.profile_callback()


Common.assignment()




Hello, my name is Common(profile_dir='directory')
